In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
986077,2021-02-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
986078,2021-02-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
986079,2021-02-12,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
986080,2021-02-13,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
26863,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26865,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26867,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26869,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26871,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
611906,2021-02-10,Arkansas,Arkansas,5001,1908,37.00,5000,Arkansas,AR,Arkansas,State,3017804
611908,2021-02-11,Arkansas,Arkansas,5001,1914,37.00,5000,Arkansas,AR,Arkansas,State,3017804
611910,2021-02-12,Arkansas,Arkansas,5001,1917,37.00,5000,Arkansas,AR,Arkansas,State,3017804
611912,2021-02-13,Arkansas,Arkansas,5001,1919,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
986077,2021-02-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
986078,2021-02-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
986079,2021-02-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
986080,2021-02-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-14') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
390,2021-02-14,Snohomish,Washington,29705,516.00,53061,WA,County,822083,3613.38,62.77
778,2021-02-14,Cook,Illinois,465914,9690.00,17031,IL,County,5150233,9046.46,188.15
1165,2021-02-14,Orange,California,256889,3577.00,6059,CA,County,3175692,8089.23,112.64
1551,2021-02-14,Maricopa,Arizona,498297,8505.00,4013,AZ,County,4485414,11109.28,189.61
1937,2021-02-14,Los Angeles,California,1167074,19066.00,6037,CA,County,10039107,11625.28,189.92
...,...,...,...,...,...,...,...,...,...,...,...
985705,2021-02-14,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
985830,2021-02-14,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
985924,2021-02-14,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
986014,2021-02-14,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
390,2021-02-14,Snohomish,Washington,29705,516.00,53061,WA,County,822083,3613.38,62.77,62.77,3613.38
778,2021-02-14,Cook,Illinois,465914,9690.00,17031,IL,County,5150233,9046.46,188.15,188.15,9046.46
1165,2021-02-14,Orange,California,256889,3577.00,6059,CA,County,3175692,8089.23,112.64,112.64,8089.23
1551,2021-02-14,Maricopa,Arizona,498297,8505.00,4013,AZ,County,4485414,11109.28,189.61,189.61,11109.28
1937,2021-02-14,Los Angeles,California,1167074,19066.00,6037,CA,County,10039107,11625.28,189.92,189.92,11625.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
985705,2021-02-14,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
985830,2021-02-14,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
985924,2021-02-14,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
986014,2021-02-14,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
695959,2021-02-14,Clarke,Alabama,3391,46.00,1025,AL,County,23622,14355.26,194.73,194.73,14355.26,1
639689,2021-02-14,Hale,Alabama,2024,59.00,1065,AL,County,14651,13814.76,402.70,402.70,13814.76,2
501081,2021-02-14,Lowndes,Alabama,1287,44.00,1085,AL,County,9726,13232.57,452.40,452.40,13232.57,3
444516,2021-02-14,Etowah,Alabama,12954,298.00,1055,AL,County,102268,12666.72,291.39,291.39,12666.72,4
373266,2021-02-14,Franklin,Alabama,3947,75.00,1059,AL,County,31362,12585.29,239.14,239.14,12585.29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610945,2021-02-14,Sublette,Wyoming,668,7.00,56035,WY,County,9831,6794.83,71.20,71.20,6794.83,19
938409,2021-02-14,Platte,Wyoming,564,11.00,56031,WY,County,8393,6719.89,131.06,131.06,6719.89,20
843032,2021-02-14,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
758166,2021-02-14,Lincoln,Wyoming,1169,11.00,56023,WY,County,19830,5895.11,55.47,55.47,5895.11,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
501081,2021-02-14,Lowndes,Alabama,1287,44.00,1085,AL,County,9726,13232.57,452.40,452.40,13232.57,3,1
639689,2021-02-14,Hale,Alabama,2024,59.00,1065,AL,County,14651,13814.76,402.70,402.70,13814.76,2,2
443208,2021-02-14,Clay,Alabama,1393,53.00,1027,AL,County,13235,10525.12,400.45,400.45,10525.12,23,3
540198,2021-02-14,Greene,Alabama,851,32.00,1063,AL,County,8111,10491.92,394.53,394.53,10491.92,25,4
213775,2021-02-14,Walker,Alabama,6360,238.00,1127,AL,County,63521,10012.44,374.68,374.68,10012.44,32,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538897,2021-02-14,Hot Springs,Wyoming,358,3.00,56017,WY,County,4413,8112.40,67.98,67.98,8112.40,15,19
739404,2021-02-14,Uinta,Wyoming,1994,12.00,56041,WY,County,20226,9858.60,59.33,59.33,9858.60,6,20
758166,2021-02-14,Lincoln,Wyoming,1169,11.00,56023,WY,County,19830,5895.11,55.47,55.47,5895.11,22,21
212776,2021-02-14,Teton,Wyoming,3278,8.00,56039,WY,County,23464,13970.34,34.09,34.09,13970.34,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983746,2021-02-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
983747,2021-02-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
983748,2021-02-12,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
983749,2021-02-13,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
693309,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,194.73,14355.26,47,1,2.00
693310,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,194.73,14355.26,47,1,0.00
693311,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,194.73,14355.26,47,1,1.00
693312,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,194.73,14355.26,47,1,4.00
693313,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,194.73,14355.26,47,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
770730,2021-02-10,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
770731,2021-02-11,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
770732,2021-02-12,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
770733,2021-02-13,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
499073,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13232.57,1,3,1.00,0.00
499074,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13232.57,1,3,0.00,0.00
499075,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13232.57,1,3,0.00,0.00
499076,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13232.57,1,3,0.00,0.00
499077,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13232.57,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497438,2021-02-10,Albany,Wyoming,3838,11.00,56001,WY,County,38880,9871.40,28.29,28.29,9912.55,23,5,6.00,0.00
497439,2021-02-11,Albany,Wyoming,3845,11.00,56001,WY,County,38880,9889.40,28.29,28.29,9912.55,23,5,7.00,0.00
497440,2021-02-12,Albany,Wyoming,3849,11.00,56001,WY,County,38880,9899.69,28.29,28.29,9912.55,23,5,4.00,0.00
497441,2021-02-13,Albany,Wyoming,3849,11.00,56001,WY,County,38880,9899.69,28.29,28.29,9912.55,23,5,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-14') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
255035,2021-02-14,Imperial,California,26680,583.00,6025,CA,County,181215,14722.84,321.72,321.72,14722.84,1,2,34.00,0.00
1937,2021-02-14,Los Angeles,California,1167074,19066.00,6037,CA,County,10039107,11625.28,189.92,189.92,11625.28,2,5,1780.00,82.00
445776,2021-02-14,Inyo,California,1168,34.00,6027,CA,County,18039,6474.86,188.48,188.48,6474.86,3,31,12.00,0.00
57061,2021-02-14,Stanislaus,California,54148,899.00,6099,CA,County,550660,9833.29,163.26,163.26,9833.29,4,10,123.00,0.00
70698,2021-02-14,Tulare,California,46162,691.00,6107,CA,County,466195,9901.87,148.22,148.22,9901.87,5,9,146.00,0.00
26517,2021-02-14,Riverside,California,285414,3524.00,6065,CA,County,2470546,11552.67,142.64,142.64,11552.67,6,6,0.00,0.00
540140,2021-02-14,Kings,California,21610,208.00,6031,CA,County,152940,14129.72,136.00,136.00,14129.72,7,3,25.00,0.00
404137,2021-02-14,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,8,8,0.00,0.00
45435,2021-02-14,San Joaquin,California,64969,992.00,6077,CA,County,762148,8524.46,130.16,130.16,8524.46,9,18,0.00,0.00
25827,2021-02-14,Fresno,California,92807,1291.00,6019,CA,County,999101,9289.05,129.22,129.22,9289.05,10,14,287.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
937960,2021-02-14,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,0.00,0.00
255035,2021-02-14,Imperial,California,26680,583.00,6025,CA,County,181215,14722.84,321.72,321.72,14722.84,1,2,34.00,0.00
540140,2021-02-14,Kings,California,21610,208.00,6031,CA,County,152940,14129.72,136.00,136.00,14129.72,7,3,25.00,0.00
127287,2021-02-14,San Bernardino,California,282169,2392.00,6071,CA,County,2180085,12943.03,109.72,109.72,12943.03,13,4,565.00,40.00
1937,2021-02-14,Los Angeles,California,1167074,19066.00,6037,CA,County,10039107,11625.28,189.92,189.92,11625.28,2,5,1780.00,82.00
26517,2021-02-14,Riverside,California,285414,3524.00,6065,CA,County,2470546,11552.67,142.64,142.64,11552.67,6,6,0.00,0.00
159921,2021-02-14,Kern,California,100290,687.00,6029,CA,County,900202,11140.83,76.32,76.32,11140.83,27,7,265.00,0.00
404137,2021-02-14,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,8,8,0.00,0.00
70698,2021-02-14,Tulare,California,46162,691.00,6107,CA,County,466195,9901.87,148.22,148.22,9901.87,5,9,146.00,0.00
57061,2021-02-14,Stanislaus,California,54148,899.00,6099,CA,County,550660,9833.29,163.26,163.26,9833.29,4,10,123.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
255035,2021-02-14,Imperial,California,26680,583.00,6025,CA,County,181215,14722.84,321.72,321.72,14722.84,1,2,34.00,0.00
1937,2021-02-14,Los Angeles,California,1167074,19066.00,6037,CA,County,10039107,11625.28,189.92,189.92,11625.28,2,5,1780.00,82.00
445776,2021-02-14,Inyo,California,1168,34.00,6027,CA,County,18039,6474.86,188.48,188.48,6474.86,3,31,12.00,0.00
57061,2021-02-14,Stanislaus,California,54148,899.00,6099,CA,County,550660,9833.29,163.26,163.26,9833.29,4,10,123.00,0.00
70698,2021-02-14,Tulare,California,46162,691.00,6107,CA,County,466195,9901.87,148.22,148.22,9901.87,5,9,146.00,0.00
26517,2021-02-14,Riverside,California,285414,3524.00,6065,CA,County,2470546,11552.67,142.64,142.64,11552.67,6,6,0.00,0.00
540140,2021-02-14,Kings,California,21610,208.00,6031,CA,County,152940,14129.72,136.00,136.00,14129.72,7,3,25.00,0.00
404137,2021-02-14,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,8,8,0.00,0.00
45435,2021-02-14,San Joaquin,California,64969,992.00,6077,CA,County,762148,8524.46,130.16,130.16,8524.46,9,18,0.00,0.00
25827,2021-02-14,Fresno,California,92807,1291.00,6019,CA,County,999101,9289.05,129.22,129.22,9289.05,10,14,287.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14722.84,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14722.84,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14722.84,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14722.84,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,321.72,14722.84,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3332,02/10/21,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,6.00,1.00
3333,02/11/21,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,0.00,0.00
3334,02/12/21,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,0.00,0.00
3335,02/13/21,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14722.84,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14722.84,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14722.84,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14722.84,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,321.72,14722.84,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3406,02/10/21,Fresno,California,91730,1258.00,6019,CA,County,999101,9181.25,125.91,129.22,9289.05,10,14,300.00,4.00
3407,02/11/21,Fresno,California,91876,1283.00,6019,CA,County,999101,9195.87,128.42,129.22,9289.05,10,14,146.00,25.00
3408,02/12/21,Fresno,California,92168,1291.00,6019,CA,County,999101,9225.09,129.22,129.22,9289.05,10,14,292.00,8.00
3409,02/13/21,Fresno,California,92520,1291.00,6019,CA,County,999101,9260.33,129.22,129.22,9289.05,10,14,352.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)